# Planning-Lab Lesson 1: Uninformed Search Strategies

In this first lesson we will work on Uninformed Search. 

### Maze Environments
The environments used is **SmallMaze** (visible in the figure).

<img src="images/maze.png" width="300">

The agent starts in cell $(0, 2)$ and has to reach the treasure in $(4, 3)$.

In order to use the environment we need first to import the packages of OpenAI Gym. Notice that due to the structure of this repository, we need to add the parent directory to the path

In [1]:
import os, sys, time, math

module_path = os.path.abspath(os.path.join('../tools'))
if module_path not in sys.path:
    sys.path.append(module_path)

from utils.ai_lab_functions import *
import gym, envs

## Assignment 1: Breadth-First Search (BFS)

Your first assignment is to implement the BFS algorithm on SmallMaze. In particular, you should implement both *tree search* and *graph search* versions of BFS that the generic bfs will call. 

The results returned by your **BFS** must be in the following form (path, time_cost, space_cost), more in detail:

- **path** - a tuple of state identifiers forming a path from the start state to the goal state. None if no solution is found.
- **time_cost** - the number of nodes checked during the exploration.
- **space_cost** - the maximum number of nodes in memory simultaneously.

**For the time_cost, we consider a node checked after its generation!**

After the correctness of your implementations has been assessed, you can run the algorithms on the **SmallMaze** environment.

Functions to implement:

- BFS_TreeSearch(problem)
- BFS_GraphSearch(problem)

Function **build_path(node)** can be used to return a tuple of states from the root node (excluded) to another node by following parent links.

Here is the pseudo-code from the book **Artificial Intelligence: A Modern Approach** for *Graph Search* and *Tree Search*:

<img src="images/tree-graph-search.png" width="600">

Here is the pseudo-code from the book **Artificial Intelligence: A Modern Approach** for the *BFS* algorithm, note that it refers to the implementation of the *Graph Search Version*:

<img src="images/BFS2.png" width="600">

**The next two functions have to be implemented:**

In [2]:
def BFS_TreeSearch(problem):
    """
    Tree Search BFS
    
    Args:
        problem: OpenAI Gym environment
        
    Returns:
        (path, time_cost, space_cost): solution as a path and stats.
    """
    
    # initialize the root node and the time and space cost
    node = Node(problem.startstate, None)
    time_cost = 1
    space_cost = 1

    # check the start state is the goal state
    if node.state == problem.goalstate:
        return build_path(node), time_cost, space_cost
    
     
    # initialize the frontier as a queue    
    frontier = NodeQueue()
    frontier.add(node) # add the root node in the queue
    
    while not frontier.is_empty(): #finche ho ancora nodi da controllare
        head = frontier.remove() # removes the first node (FIFO)
        
        # check the actions available from the current state
        for action in range(problem.action_space.n):
            # for every action, sample the next state
            next_state = problem.sample(head.state,action)
            # add child in the queue increment the time_cost and check if goalstate
            child = Node(next_state,head) 
            time_cost +=1
            if child.state == problem.goalstate:
                # if the child is the goal state, return the path and the stats
                return build_path(child), time_cost, space_cost
            frontier.add(child) 
            
        space_cost = max(space_cost,len(frontier))
    
    return None, time_cost, space_cost


In [3]:
def BFS_GraphSearch(problem):
    """
    Graph Search BFS
    
    Args:
        problem: OpenAI Gym environment
        
    Returns:
        (path, time_cost, space_cost): solution as a path and stats.
    """
    
    # initialize the root node and the time and space cost
    node = Node(problem.startstate, None)
    time_cost = 1
    space_cost = 1
    
    # check the start state is the goal state
    if node.state == problem.goalstate:
        return build_path(node), time_cost, space_cost
    
    # initialize the frontier as a queue
    frontier = NodeQueue()
    frontier.add(node)
    # in graph search we need to keep track of the explored nodes
    explored = []
    
    # until the frontier is not empty
    while not frontier.is_empty():
        # remove the first node and add it to the explored list
        current = frontier.remove()
        explored.append(current.state)
        
        # check the actions available from the current state
        for action in range(problem.action_space.n):

            # for every action, sample the next state
            next_node = problem.sample(current.state,action) 
            # add child in the queue increment the time_cost and check if goalstate
            child = Node(next_node,current) 
            time_cost +=1

            # if the child is not in the explored list and not in the frontier
            if child.state not in explored and child.state not in frontier:
                # if the child is the goal state, return the path and the stats
                if problem.goalstate == child.state:
                    return build_path(child), time_cost, space_cost
                # if the child is not the goal state, add it to the frontier 
                frontier.add(child)
            
                
        space_cost = max(space_cost,len(frontier)+len(explored))
    
    return None, time_cost, space_cost        

**The following code calls your tree search and graph search version of BFS and checks the results**

In [4]:
envname = "SmallMaze-v0"
environment = gym.make(envname)

solution_ts, time_ts, memory_ts = BFS_TreeSearch(environment)
solution_gs, time_gs, memory_gs = BFS_GraphSearch(environment)

results = CheckResult_L1A1([solution_ts, time_ts, memory_ts], [solution_gs, time_gs, memory_gs], environment)
results.check_sol_ts()
results.check_sol_gs()

##########################################
#######  BFS TREE SEARCH PROBLEM  ########
##########################################
Your solution: [(0, 1), (0, 0), (1, 0), (2, 0), (3, 0), (4, 0), (4, 1), (4, 2), (4, 3)]
N° of nodes explored: 103723
Max n° of nodes in memory: 77791

===> Your solution is correct!

##########################################
#######  BFS Graph SEARCH PROBLEM  #######
##########################################
Solution: [(0, 1), (0, 0), (1, 0), (2, 0), (3, 0), (4, 0), (4, 1), (4, 2), (4, 3)]
N° of nodes explored: 59
Max n° of nodes in memory: 15

===> Your solution is correct!



## Assignment 2:  Depth-Limited Search (DLS) and Iterative Deepening depth-first Search (IDS)

Your second assignment is to implement the IDS algorithm on SmallMaze. 
In particular, you are required to implement *DLS* in the *graph search* version, *DLS* in the *tree search* version, and the final *Iterative_DLS*.

Similarly to assignment 1, the results returned by your ids must be in the following form (path, Time Cost, Space Cost) described above. After the correctness of your implementations has been assessed, you can run the algorithms on the **SmallMaze** environment.

Functions to implement:

- Recursive_DLS_TreeSearch(node, problem, limit)
- Recursive_DLS_GraphSearch(node, problem, limit, explored)
- IDS(problem)

Function **build_path(node)** can be used to return a tuple of states from the root node (excluded) to another node by following parent links.

Here is the pseudo-code from the book **Artificial Intelligence: A Modern Approach** for the *Depth-Limited Search (Tree Search Version)* and *Iterative deepening depth-first search (Tree Search Version)*:
<img src="images/dls.png" width="600">
<img src="images/ids.png" width="600">

#### Note that Node() has a depthcost variable that represents the depth of the node in the search tree. This variable is automatically set by the Node constructor: if the root node has a depthcost of 0, its children will have a depthcost increased by 1. An example is shown below. The depthcost is useful to compute the space cost for IDS. An example is shown below:

In [5]:
start = environment.startstate
root = Node(start)  # parent = None and depthcost = 0 as default
child = Node(environment.sample(start, 0), root) # the depthcost is set automatically in the Node constructor
print("Root depthcost: {}\tChild depthcost: {}".format(root.depthcost, child.depthcost))

Root depthcost: 0	Child depthcost: 1


In [6]:
def DLS(problem, limit, RDLS_Function):
    """
    DLS
    
    Args:
        problem: OpenAI Gym environment
        limit: depth limit for the exploration, negative number means 'no limit'
        
    Returns:
        (path, time_cost, space_cost): solution as a path and stats.
    """
        
    node = Node(problem.startstate, None)
    return RDLS_Function(node, problem, limit, set())

**The next two functions have to be implemented:**

In [7]:
def Recursive_DLS_GraphSearch(node, problem, limit, explored):
    """
    Recursive DLS
    
    Args:
        node: node to explore
        problem: OpenAI Gym environment
        limit: depth limit for the exploration, negative number means 'no limit'
        explored: completely explored nodes
        
    Returns:
        (path, time_cost, space_cost): solution as a path and stats.
    """
        
    # Check if the current node is the goal state 
    if problem.goalstate == node.state:
        return build_path(node),1,node.depthcost
    
    # Check if the current depth is equal to the limit
    if limit == 0:
        return "cut_off",1,node.depthcost
    
    # Add the current node to the explored list
    explored.add(node.state)
    space_cost = node.depthcost
    time_cost = 1 
    cut_off_occurred = False
    
    # Check the actions available from the current state
    for action in range(problem.action_space.n):
        # For every action, sample the next state
        next_node = problem.sample(node.state,action)
        # Generate child node 
        child = Node(next_node,node,node.depthcost+1) # generate child node
        # Check if the child is not in the explored list
        if not child.state in explored:
            # Recursive call to the function
            path,time,space = Recursive_DLS_GraphSearch(child,problem,limit-1,explored)
            time_cost += time
            space_cost = max(space,space_cost)
            
            # Check if the path is a cut_off or a failure
            if path == "cut_off":
                cut_off_occurred = True

            elif path != "failure":
                return path,time_cost,space_cost
    # Check if the cut_off_occurred            
    if cut_off_occurred:
        return "cut_off", time_cost, space_cost
        
    # return failure is no solution is found
    return "failure", time_cost, space_cost

In [8]:
def Recursive_DLS_TreeSearch(node, problem, limit, explored=None):
    """
    This function is similar to the previous one, but it is a 
    tree search version of the Recursive Depth-Limited Search (DLS) algorithm. 
    The main difference is that it doesn't take in the set of explored nodes 
    as a parameter and doesn't use it within the function. 
    The function uses a recursive approach to explore the tree, 
    where it first checks if the current node is the goal state, 
    if so it returns the path to the goal state, 
    otherwise it checks if the current depth is equal to the limit, 
    if so it returns a "cut_off" message, if not it continues to explore the children
    of the current node. If the algorithm reaches a point where it can 
    no longer continue exploring, it returns a "failure" message. 
        
    Args:
        node: node to explore
        problem: OpenAI Gym environment
        limit: depth limit for the exploration, negative number means 'no limit'
        
    Returns:
        (path, time_cost, space_cost): solution as a path and stats.
    """
    

    # Check if the current node is the goal state
    if problem.goalstate == node.state:
        return build_path(node),1,len(explored)

    # Check if the current depth is equal to the limit
    if limit == 0:
        return "cut_off",1,node.depthcost
    
    space_cost = node.depthcost
    time_cost = 1 
    cut_off_occurred = False
    
    # Check the actions available from the current state
    for action in range(problem.action_space.n):
        next_node = problem.sample(node.state,action) 
        # Generate child node
        child = Node(next_node,node,node.depthcost+1)
        
        path,time,space = Recursive_DLS_TreeSearch(child,problem,limit-1,explored)
        
        time_cost += time
        space_cost = max(space,space_cost)
            
        if path == "cut_off":
            cut_off_occurred = True
        elif path != "failure":
            return path,time_cost,space_cost
            
    if cut_off_occurred:
        return "cut_off", time_cost, space_cost
    return "failure", time_cost, space_cost

In [9]:
def IDS(problem, DLS_Function):
    """
    Iteartive_DLS DLS
    
    Args:
        problem: OpenAI Gym environment
        
    Returns:
        (path, time_cost, space_cost): solution as a path and stats.
    """
        
    total_time_cost = 0
    total_space_cost = 1
    for i in zero_to_infinity():
        path,time,space = DLS(problem,i,DLS_Function)
        total_space_cost = max(total_space_cost, space)
        total_time_cost += time
        
        if path != "cut_off":
            return path, total_time_cost, total_space_cost, i

**The following code calls your version of IDS and checks the results:**

In [10]:
envname = "SmallMaze-v0"
environment = gym.make(envname)

solution_ts, time_ts, memory_ts, iterations_ts = IDS(environment, Recursive_DLS_TreeSearch)
solution_gs, time_gs, memory_gs, iterations_gs = IDS(environment, Recursive_DLS_GraphSearch)

results = CheckResult_L1A2([solution_ts, time_ts, memory_ts, iterations_ts], [solution_gs, time_gs, memory_gs, iterations_gs], environment)
results.check_sol_ts()
results.check_sol_gs()

##########################################
#######  IDS TREE SEARCH PROBLEM  ########
##########################################
Necessary Iterations: 9
Your solution: [(0, 1), (0, 0), (1, 0), (2, 0), (3, 0), (4, 0), (4, 1), (4, 2), (4, 3)]
N° of nodes explored: 138298
Max n° of nodes in memory: 9

===> Your solution is correct!

##########################################
#######  IDS GRAPH SEARCH PROBLEM  #######
##########################################
Necessary Iterations: 11
Solution: [(0, 1), (0, 0), (1, 0), (1, 1), (2, 1), (2, 0), (3, 0), (4, 0), (4, 1), (4, 2), (4, 3)]
N° of nodes explored: 132
Max n° of nodes in memory: 11

===> Your solution is correct!



### Discussion

Now that you have correctly implemented both BFS and IDS what can you say about the solutions they compute? Are there significant differences in the stats?